In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


In [2]:
def loadData():
    with np.load("notMNIST.npz") as data:
        Data, Target = data["images"], data["labels"]
        np.random.seed(521)
        randIndx = np.arange(len(Data))
        np.random.shuffle(randIndx)
        Data = Data[randIndx] / 255.0
        Target = Target[randIndx]
        trainData, trainTarget = Data[:10000], Target[:10000]
        validData, validTarget = Data[10000:16000], Target[10000:16000]
        testData, testTarget = Data[16000:], Target[16000:]
    return trainData, validData, testData, trainTarget, validTarget, validTarget


In [3]:
def convertOneHot(trainTarget, validTarget, testTarget):
    newtrain = np.zeros((trainTarget.shape[0], 10))
    newvalid = np.zeros((validTarget.shape[0], 10))
    newtest = np.zeros((testTarget.shape[0], 10))

    for item in range(0, trainTarget.shape[0]):
        newtrain[item][trainTarget[item]] = 1
    for item in range(0, validTarget.shape[0]):
        newvalid[item][validTarget[item]] = 1
    for item in range(0, testTarget.shape[0]):
        newtest[item][testTarget[item]] = 1
    return newtrain, newvalid, newtest


In [4]:
def shuffle(trainData, trainTarget):
    np.random.seed(421)
    randIndx = np.arange(len(trainData))
    target = trainTarget
    np.random.shuffle(randIndx)
    data, target = trainData[randIndx], target[randIndx]
    return data, target



In [5]:
def relu(x) : 
    return np.maximum(x,0)

In [10]:
def softmax(z) :
    N = z.shape
    e_pow = np.exp(z)
    summation = np.sum(e_pow)
    out = np.divide(e_pow,summation)
    return out

In [13]:
def computeLayer(X,W,b) :
    return np.dot(W.T,x) + b

In [22]:
def CE(target, prediction):
    N = target.shape
    #check assignment anouncement
    log_term = np.log(prediction)
    CE_out = np.multiply(target,log_term)
    CE_out = (np.sum(CE_out)) / N
    return CE_out

In [24]:
train_x, train_y, test_x, test_y , valid_x, valid_y = loadData()
#train_y, valid_y, test_y = convertOneHot(train_y,valid_y,test_y)


In [25]:
def gradCE(target, prediction):

    # TODO


SyntaxError: unexpected EOF while parsing (<ipython-input-25-c12b5ed612cf>, line 3)